<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/SOES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download es_core_news_md

     |████████████████████████████████| 78.4MB 1.2MB/s 
  Created wheel for es-core-news-md: filename=es_core_news_md-2.2.5-cp36-none-any.whl size=79649483 sha256=684009d5e7d4bbbb5aa75c47cecc99d2f7177544576271c5ca1a5f22c825688d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3aoh22ad/wheels/b7/bb/a3/29ab5cf80c2c0a8fa0f2af8402fdace3f159e8265f0fdcbcdb
Successfully built es-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
from time import time
start_nb = time()

start = time()
import pandas as pd
from nltk import SnowballStemmer
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

import es_core_news_md
nlp = es_core_news_md.load()

es_stops = set(stopwords.words('spanish'))
Snowball_stemmer = SnowballStemmer('spanish')

print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 17.71 segundos


# PRE-PROCESSED

### DATASET

In [5]:
start = time()
dataset = pd.read_csv('//content/gdrive/My Drive/df.csv',
                      sep=';', encoding='utf-8-sig')
dataset_size = len(dataset)


print('Dataset generado. %.2f segundos' % (time() - start))
dataset.head()

Dataset generado. 0.22 segundos


,Id,Title,Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...


### LEMATIZACIÓN

In [6]:
def preprocess_lemma(doc):
    separator = ' '
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = separator.join(doc)
    doc = nlp(doc)
    doc = [w.lemma_ for w in doc]
    return doc

In [7]:
start = time()
dataset['Lemma_Title'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Title']), axis=1)
dataset['Lemma_Body'] = dataset.apply(
        lambda dataset: preprocess_lemma(dataset['Body']), axis=1)
print('Dataset generado. %.2f segundos' % (time() - start))
dataset.iloc[:,[3,4]]

Dataset generado. 28.45 segundos


,Lemma_Title,Lemma_Body
0,"[debugear, código, django, utilizar, docker]","[contar, realizar, proyectar, django, utilizar..."
1,"[mostrar, error, usar, supervisor, digital, oc...","[configurar, ruta, sudar, nano, program, factu..."
2,"[ejecutar, navegador, pantalla, completo, inic...","[realizar, proyectar, domotico, siguiente, com..."
3,"[paginar, web, blanco]","[bueno, dia, paginar, web, salir, blanco, algu..."
4,"[obtener, error, skipped, frames, the, applica...","[trabajar, android, studio, intentar, hacer, c..."
...,...,...
1079,"[conexion, remoto, sql, server, linux, mint]","[instalar, sql, server, linux, mint, basar, da..."
1080,"[crear, basar, dato, centos, funcionar]","[utilizar, centos, seguir, instrucción, sitiar..."
1081,"[hacer, respaldar, basar, dato, terminal]","[basar, dato, milisegundo, sql, server, expres..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollar, utilizar, cámara, es..."


### STEMMER

In [8]:
def preprocess_stem(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    doc = [Snowball_stemmer.stem(w) for w in doc]  # Extraer raices o stems (Stemming)
    return doc

In [9]:
start = time()
dataset['Stem_Title'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Title']), axis=1)
dataset['Stem_Body'] = dataset.apply(
        lambda dataset: preprocess_stem(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[5,6]]

Preprocesado completo. 3.60 segundos


,Stem_Title,Stem_Body
0,"[debug, codig, djang, utiliz, dock]","[cuent, realiz, proyect, djang, utiliz, pychar..."
1,"[muestr, error, usand, supervisor, digital, oc...","[configur, rut, sud, nan, program, facturaempr..."
2,"[ejecut, naveg, pantall, complet, inici, linux]","[realiz, proyect, domot, siguient, component, ..."
3,"[pagin, web, blanc]","[buen, dia, pagin, web, sal, blanc, algui, pod..."
4,"[obteng, error, skipp, fram, the, application,...","[trabaj, andro, studi, intent, hac, conexion, ..."
...,...,...
1079,"[conexion, remot, sql, serv, linux, mint]","[instal, sql, serv, linux, mint, bas, dat, cre..."
1080,"[cre, bas, dat, cent, funcion]","[utiliz, cent, segu, instruccion, siti, micros..."
1081,"[hac, respald, bas, dat, terminal]","[bas, dat, ms, sql, serv, express, ubuntu, adm..."
1082,"[aplic, windows, linux, win]","[aplic, desarroll, utiliz, cam, escan, lector,..."


### PRE-PROCESADO (SOLO STOPWORD & SIGNOS DE PUNTUACIÓN)

In [10]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [11]:
start = time()
dataset['Processed_Title'] = dataset.apply(
        lambda dataset: preprocess(dataset['Title']), axis=1)
dataset['Processed_Body'] = dataset.apply(
        lambda dataset: preprocess(dataset['Body']), axis=1)
print('Preprocesado completo. %.2f segundos' % (time() - start))
dataset.iloc[:,[7,8]]

Preprocesado completo. 1.59 segundos


,Processed_Title,Processed_Body
0,"[debugear, código, django, utilizando, docker]","[cuento, realizando, proyecto, django, utilizo..."
1,"[muestra, error, usando, supervisor, digital, ...","[configurar, ruta, sudo, nano, program, factur..."
2,"[ejecutar, navegador, pantalla, completa, inic...","[realizando, proyecto, domotico, siguientes, c..."
3,"[pagina, web, blanco]","[buen, dia, pagina, web, sale, blanco, alguien..."
4,"[obtengo, error, skipped, frames, the, applica...","[trabajando, android, studio, intentando, hace..."
...,...,...
1079,"[conexion, remota, sql, server, linux, mint]","[instalado, sql, server, linux, mint, base, da..."
1080,"[crear, base, datos, centos, funciona]","[utilizando, centos, seguí, instrucciones, sit..."
1081,"[hacer, respaldo, base, datos, terminal]","[base, datos, ms, sql, server, express, ubuntu..."
1082,"[aplicación, windows, linux, wine]","[aplicación, desarrollada, utiliza, cámara, es..."


# WORD MOVER DISTANCE (WMD)

In [12]:
from sklearn.model_selection import train_test_split 

### WMD - LEMMA

In [13]:
train_l, test_l = train_test_split(dataset['Lemma_Title'], test_size = 0.20, shuffle = False)

print("(Lemma) Ejemplos usados para entrenar: ", len(train_l))
print("(Lemma) Ejemplos usados para test: ", len(test_l))

(Lemma) Ejemplos usados para entrenar:  867
(Lemma) Ejemplos usados para test:  217


In [14]:
test_l

867                 [backup, postgresql, error, password]
868     [ejecutar, sentenciar, postgresql, líneo, coma...
869                          [raspberry, pi, kali, linux]
870     [existir, alguno, alternativo, raspberry, pi, ...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remoto, sql, server, linux, mint]
1080              [crear, basar, dato, centos, funcionar]
1081            [hacer, respaldar, basar, dato, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicitar, servidor, usuario, ftp,...
Name: Lemma_Title, Length: 217, dtype: object

In [15]:
lemma_title = dataset['Lemma_Title'].to_list()

title = dataset['Title'].to_list()

In [16]:
from gensim.models import Word2Vec

model_l = Word2Vec(lemma_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_l = WmdSimilarity(test_l, model_l, num_best)
print(instancia_l)

WmdSimilarity<217 docs, 100 features>


In [17]:
start = time()

msj_l = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_l = preprocess_lemma(msj_l)

sim_l = instancia_l[bus_l]
print(len(sim_l))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.09 seconds to run.


In [18]:
print ('Busqueda: ', msj_l)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_l[i][0]],'---> sim = %.4f' % sim_l[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
grep combinado con echo ---> sim = 0.5392
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5345
¿Como agregar varios puertos al mismo contenedor de Docker? ---> sim = 0.5268
Como ejecutar mas de un archivo que esta en un parámetro de jenkins ---> sim = 0.5266
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.5264
¿como puedo solucionar esto kernel panic not syncing vfs unable en parrot? ---> sim = 0.5242
Cambiar permisos de carpetas compartidas con docker-compose ---> sim = 0.5242
Problema con git en local ---> sim = 0.5236
¿Como verificar si una cadena ingresada se encuentra dentro de un rango numérico? ---> sim = 0.5224
Pantalla completa Kali Linux VirtualBox? ---> sim = 0.5216


### WMD - STEMMER

In [19]:
train_s, test_s = train_test_split(dataset['Stem_Title'], test_size = 0.20, shuffle = False)

print("(Stemm) Ejemplos usados para entrenar: ", len(train_s))
print("(Stemm) Ejemplos usados para test: ", len(test_s))

(Stemm) Ejemplos usados para entrenar:  867
(Stemm) Ejemplos usados para test:  217


In [20]:
stemm_title = dataset['Stem_Title'].to_list()

title = dataset['Title'].to_list()

In [21]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_s = Word2Vec(stemm_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_s = WmdSimilarity(test_s, model_s, num_best)
print(instancia_s)

WmdSimilarity<217 docs, 100 features>


In [22]:
start = time()

msj_s = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_s = preprocess_stem(msj_s)

sim_s = instancia_s[bus_s] 
print(len(sim_s))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.08 seconds to run.


In [23]:
print ('Busqueda: ', msj_s)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_s[i][0]],'---> sim = %.4f' % sim_s[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
¿como puedo solucionar esto kernel panic not syncing vfs unable en parrot? ---> sim = 0.6256
Cambiar permisos de carpetas compartidas con docker-compose ---> sim = 0.6256
¿Cómo mostrar en pantalla solo los archivos de noviembre y diciembre para luego hacerle un grep? ---> sim = 0.6219
Pantalla completa Kali Linux VirtualBox? ---> sim = 0.6186
Problema server vsftpd linux ---> sim = 0.6104
moodle apache centos 7 ---> sim = 0.6103
¿Como hago para reinstalar libbz2.so.1.0? ---> sim = 0.5790
Obtengo el error I/Choreographer: Skipped 110 frames! The application may be doing too much work on its main thread ---> sim = 0.5634
problemas con la configuracion de GitLab, netbeans y linux ---> sim = 0.5616
Problema con git en local ---> sim = 0.5512


### WMD - TEXT

In [24]:
def preprocess(doc):
    label = re.compile('<.*?>')
    doc = re.sub(label, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [25]:
train_p, test_p = train_test_split(dataset['Processed_Title'], test_size = 0.20, shuffle = False)

print("Ejemplos usados para entrenar: ", len(train_p))
print("Ejemplos usados para test: ", len(test_p))

Ejemplos usados para entrenar:  867
Ejemplos usados para test:  217


In [26]:
test_p

867                 [backup, postgresql, error, password]
868     [ejecutar, sentencias, postgresql, línea, coma...
869                          [raspberry, pi, kali, linux]
870     [existe, alguna, alternativa, raspberry, pi, c...
871     [problema, instalar, drive, adaptador, usb, wi...
                              ...                        
1079         [conexion, remota, sql, server, linux, mint]
1080               [crear, base, datos, centos, funciona]
1081             [hacer, respaldo, base, datos, terminal]
1082                   [aplicación, windows, linux, wine]
1083    [wordpress, solicita, servidor, usuario, ftp, ...
Name: Processed_Title, Length: 217, dtype: object

In [27]:
processed_title = dataset['Processed_Title'].to_list()

title = dataset['Title'].to_list()

In [28]:
from gensim.models import Word2Vec
# Train Word2Vec on all the restaurants.
model_p = Word2Vec(processed_title, workers=3, size=100)

# Inicialización de WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instancia_p = WmdSimilarity(train_p, model_p, num_best=10)
print(instancia_p)

WmdSimilarity<867 docs, 100 features>


In [29]:
start = time()

msj_p = 'Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian'
bus_p = preprocess(msj_p)

sim_p = instancia_p[bus_p]
print(len(sim_p))
print ('Cell took %.2f seconds to run.' %(time() - start))

10
Cell took 0.28 seconds to run.


In [30]:
print ('Busqueda: ', msj_p)
print("-------------------------------------------------------")
for i in range(num_best): # (num_best)
    print (title[sim_p[i][0]],'---> sim = %.4f' % sim_p[i][1])

Busqueda:  Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian
-------------------------------------------------------
Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian ---> sim = 1.0000
Tengo un error constante en Debian cuando quiero actualizar mi sistema ---> sim = 0.6554
I/O Error: Sistema de archivos de solo lectura en SSD (XUBUNTU) ---> sim = 0.6034
Error al instalar php-db en Debian 10 Buster ---> sim = 0.5989
Error: spawn EACCES -- Derivados debian -- ionic ---> sim = 0.5942
Error instalar xampp en linux ---> sim = 0.5116
¿como puedo obtener informacion del sistema emulando /proc/net? ---> sim = 0.5112
¿Cómo montar un disco duro NTFS en Linux (Debian)? ---> sim = 0.5110
Problema con linux en instalación /permisos ---> sim = 0.5106
Error al conectar dispositivo HDMI . No input driver specified, ignoring this device ---> sim = 0.5103


In [96]:
dataset.iloc[:,[3,5,7]]
dataset.to_csv('ultimate.csv', sep=';', encoding='utf-8-sig')

# VECTOR SPACE MODEL (VSM)

In [32]:
from collections import Counter
import numpy as np
import math

p_title = dataset['Processed_Title'].to_list()
p_body = dataset['Processed_Body'].to_list()

N = len(dataset) # N = Cantidad de documentos
alpha = 0.3
DF = {}

for i in range(N):
  for w in p_title[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}
  for w in p_body[i]:
      try:
        DF[w].add(i)
      except:
        DF[w] = {i}

for i in DF:
        DF[i] = len(DF[i])

total_vocab_size = len(DF)
total_vocab = [x for x in DF]

In [33]:
def doc_freq(DF, word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [34]:
    start = time()

    doc = 0
    tf_idf_title = {}

    for i in range(N):
        tokens = p_body[i]
        counter = Counter(tokens + p_title[i])
        words_count = len(tokens + p_title[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = np.log((N + 1) / (df + 1))
            tf_idf_title[doc, token] = tf * idf
        doc += 1

    doc = 0
    tf_idf_body = {}

    for i in range(N):
        tokens = p_title[i]
        counter = Counter(tokens + p_body[i])
        words_count = len(tokens + p_body[i])
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            # numerator is added 1 to avoid negative values
            idf = np.log((N + 1) / (df + 1))  
            tf_idf_body[doc, token] = tf * idf
        doc += 1

    for i in tf_idf_title:
        tf_idf_title[i] *= alpha

    for i in tf_idf_body:
        tf_idf_title[i] = tf_idf_body[i]

    # D = Matriz TF-IDF de terminos de documentos
    D = np.zeros((N, total_vocab_size)) 
    for i in tf_idf_title:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf_title[i]
        except:
            pass

    print('Matriz TF-IDF de terminos de documentos generada. %.2f segundos' % (time() - start))
    print(D[0])
    print(D[1])

Matriz TF-IDF de terminos de documentos generada. 2.43 segundos
[0.16144072 0.05125391 0.12287463 ... 0.         0.         0.        ]
[0. 0. 0. ... 0. 0. 0.]


In [35]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [36]:
def print_doc(dataset, id):
    with pd.option_context('display.max_colwidth', 500):
        print(dataset.iloc[[id], [0,1]])

In [205]:
start = time()

def gen_vector(tokens):
        Q = np.zeros((len(total_vocab)))
        counter = Counter(tokens)
        words_count = len(tokens)
        for token in np.unique(tokens):
            tf = counter[token] / words_count
            df = doc_freq(DF, token)
            idf = math.log((N + 1) / (df + 1))
            try:
                ind = total_vocab.index(token)
                Q[ind] = tf * idf
            except:
                pass
        return Q

def cosine_similarity(k, query):
        tokens = preprocess(query)
        print("\nQuery:", query)
        d_cosines = []
        query_vector = gen_vector(tokens)
        for d in D:
            d_cosines.append(cosine_sim(query_vector, d))
        index_sort = np.array(d_cosines).argsort()[-k:][::-1]
        value_array = np.array(d_cosines)
        value_sort = np.sort(value_array)[-k:][::-1]
        res = []
        for i in range(k):
            res.append((index_sort[i], value_sort[i]))
        return res

results = cosine_similarity(5, "comando sort linux")

print("\n")
for i in range(len(results)):
  print_doc(dataset, results[i][0])
  print("sim: ",results[i]) 
  print("-------------------------------------------------------")

print('Tiempo de ejecución: %.2f segundos' % (time() - start))


Query: comando sort linux


         Id               Title
730  333044  Comando sort linux
sim:  (730, 0.8811562305046355)
-------------------------------------------------------
        Id                                                                               Title
661  91952  ¿Cómo funciona este código para ver intentos de conexión a tu servidor en CentOs7?
sim:  (661, 0.3191679678748724)
-------------------------------------------------------
         Id                         Title
323  212579  Comando equivalente en Linux
sim:  (323, 0.13800630021480367)
-------------------------------------------------------
         Id                                           Title
353  202549  comando para interfaz para cambiar ip en linux
sim:  (353, 0.11402418179538279)
-------------------------------------------------------
         Id                                                  Title
137  281456  ¿Como ejecuto un comando como root desde un .desktop?
sim:  (137, 0.11289343858

# TEST

In [97]:
start = time()
data = pd.read_csv('//content/gdrive/My Drive/ultimate.csv',
                      sep=';', encoding='utf-8-sig')


print('Dataset generado. %.2f segundos' % (time() - start))
data.head()

Dataset generado. 0.18 segundos


,Unnamed: 0,Id,Title,Body,Lemma_Title,Lemma_Body,Stem_Title,Stem_Body,Processed_Title,Processed_Body
0,0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...,debugear código django utilizar docker,contar realizar proyectar django utilizar pych...,debug codig djang utiliz dock,cuent realiz proyect djang utiliz pycharm wind...,debugear código django utilizando docker,cuento realizando proyecto django utilizo pych...
1,1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...,mostrar error usar supervisor digital ocean op...,configurar ruta sudar nano program facturaempr...,muestr error usand supervisor digital oce opci...,configur rut sud nan program facturaempres eje...,muestra error usando supervisor digital ocean ...,configurar ruta sudo nano program facturaempre...
2,2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...,ejecutar navegador pantalla completo iniciar l...,realizar proyectar domotico siguiente componen...,ejecut naveg pantall complet inici linux,realiz proyect domot siguient component raspbe...,ejecutar navegador pantalla completa iniciar l...,realizando proyecto domotico siguientes compon...
3,3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...,paginar web blanco,bueno dia paginar web salir blanco alguien pod...,pagin web blanc,buen dia pagin web sal blanc algui podri ayud ...,pagina web blanco,buen dia pagina web sale blanco alguien podria...
4,4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...,obtener error skipped frames the application m...,trabajar android studio intentar hacer conexio...,obteng error skipp fram the application may be...,trabaj andro studi intent hac conexion maquin ...,obtengo error skipped frames the application m...,trabajando android studio intentando hacer con...


In [165]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import string

count = data['Processed_Title'].to_list()
# Create the Document Term Matrix
vectorizer = CountVectorizer()

In [166]:
sparse_matrix = vectorizer.fit_transform(count)
print(sparse_matrix[0:1])


  (0, 423)	1
  (0, 402)	1
  (0, 527)	1
  (0, 1771)	1
  (0, 530)	1


In [170]:
doc_term_matrix = sparse_matrix.todense()
doc_term_matrix

df = pd.DataFrame(doc_term_matrix, 
                  columns=vectorizer.get_feature_names())
df[:1]

,abierto,abrir,abro,absoluta,accede,acceder,acceso,access,acción,acentos,acl,activar,actual,actualiza,actualizaciones,actualización,actualizar,actúe,adaptador,additions,addons,addres,adios,advertencias,afuera,again,agente,agrandar,agregar,agrupar,aix,aleatoria,alguna,algún,alias,almacenadas,almacenar,almohadilla,alojada,alojados,...,why,wifi,wifislax,wifit,wikidata,will,window,windows,wine,wireless,with,word,wordpress,work,workrave,world,ws,wxmaxima,xamarin,xampp,xe,xlsx,xp,xrdp,xubuntu,yes,yii,your,yum,zabbix,zip,ziparchive,zombie,zoom,zte,árbol,éste,última,último,único
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
search = []
search = input("Ingrese pregunta: ")
search = search.split()
print(search)

Ingrese pregunta: comando sort linux
['comando', 'sort', 'linux']


In [113]:
query = vectorizer.fit_transform(list(search))

print(query[:])

  (0, 0)	1
  (1, 2)	1
  (2, 1)	1


In [114]:
do_search = query.todense()
do_search

df = pd.DataFrame(do_search, 
                  columns=vectorizer.get_feature_names())
df[:2]

,comando,linux,sort
0,1,0,0
1,0,0,1


In [115]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(sparse_matrix, sparse_matrix))

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.20412415 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.20412415 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [150]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(doc_term_matrix[5], doc_term_matrix).flatten()
print(cosine_similarities)

[0. 0. 0. ... 0. 0. 0.]


In [155]:
related_docs_indices = cosine_similarities.argsort()[:-11:-1]
print(related_docs_indices)
print(len(related_docs_indices))
print(cosine_similarities[related_docs_indices])
score= cosine_similarities[related_docs_indices]

[  5 506 682 496 505 504 502 501 499 498]
10
[6. 2. 2. 1. 1. 1. 1. 1. 1. 1.]


In [156]:
for i in range(len(related_docs_indices)):
    print("score",score[i],"->",related_docs_indices[i],"--->",count[related_docs_indices[i]])

score 6.0 -> 5 ---> java android generar documento partir plantilla
score 2.0 -> 506 ---> please include gradle in your path or install android studio android studio ionic
score 2.0 -> 682 ---> puedo generar archivo posea variable quiero partir varios archivos
score 1.0 -> 496 ---> emulador android studio suena ubuntu distro
score 1.0 -> 505 ---> problema abrir proyectos android studio ubuntu
score 1.0 -> 504 ---> utilizar android studio gradle instala linux
score 1.0 -> 502 ---> puedo abrir android studio versiones linux debian
score 1.0 -> 501 ---> cambiar dirección ip dispositivo android si usando ethernet
score 1.0 -> 499 ---> ruta tarjeta removible sd card android xamarin
score 1.0 -> 498 ---> android studio mejor desarrollo aplicaciones libreria opencv


TEST - SOFT COSINE

In [53]:
from sklearn import preprocessing


array([1, 1, 2, 6])

In [ ]:
exa = data['Processed_Title'].to_list()
exa

In [225]:
le = preprocessing.LabelEncoder()
var = le.fit_transform(exa)
var

array([ 256,  641,  311, ...,  503,   48, 1080])

In [227]:
print(list(le.classes_))
# ['amsterdam', 'paris', 'tokyo']

['abrir archivo directorio', 'abrir archivo puerto especifico linux', 'abrir externa recursos ubuntu', 'abrir fichero archivo determinado programa mediante consola linux', 'abrir fichero modo lectura c', 'abrir programa compilado terminal', 'abrir puerto udp servidor debian', 'acceder archivos linux contraseña pc red', 'acceder carpeta red gdrive terminal puedo verla dolphin kubuntu', 'acceder directorio oculto linux', 'acceder mediante ip puerto redireccionar url', 'acceder root usuario baja prioridad', 'acceder servidor virtual linux docker afuera', 'acceder sitio localhost', 'acceso archivos mediante php alias directorio apache', 'acceso denegado phpmyadmin ubuntu', 'acceso servidor web local linux', 'acceso xampp internet', 'access denied for user using password', 'access denied for user using password yes', 'activar', 'actualiza programa linux terminal', 'actualizar kali', 'actualizar nodejs ubuntu', 'actualizar openssl servidor centos', 'actualizar xampp linux ubuntu', 'adaptador

In [228]:
list(le.inverse_transform([256,641]))

['debugear código django utilizando docker',
 'muestra error usando supervisor digital ocean opcion corregirlo']

In [210]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [213]:
search = preprocess("Error al automatizar la instalación de Postgresql 9.4 en sistema embebido con Debian")
print(search)


['error', 'automatizar', 'instalación', 'postgresql', 'sistema', 'embebido', 'debian']


In [214]:
question = le.fit_transform(search)
question

array([3, 0, 4, 5, 6, 2, 1])